In [ ]:
# Clean up LinkML section creation prefixes

import rdflib

def unify_ns_prefixes(
    input_turtle: str,
    output_turtle: str,
    base_uri: str = "https://example.org/ilcd/",
    old_ns_list = (
        "ILCDex:",
        "ILCDsd:",
        "ILCDlcia:",
        "ILCDmav:",
        "ILCDadmin:",
        "ILCDpi:",
    )
):
    """
    Load a Turtle file containing multiple 'ns' prefixes and unify them
    into a single 'ilcd:' prefix, rewriting URIs that match old_ns_list
    to the 'base_uri'.

    :param input_turtle: Path to the original .ttl file (with multiple prefixes)
    :param output_turtle: Path to the new .ttl file (with a single prefix "ilcd")
    :param base_uri: URI used for rewriting old prefixes (ends with '/')
    :param old_ns_list: Tuple of strings to unify. Each string is the old namespace up to but not including local name
                       E.g. "ILCDex:", "ILCDsd:", or the generated "ns1:" etc.
                       You can also match the expanded forms if you prefer.
    """
    # 1) Parse the old Turtle file into an rdflib Graph
    old_g = rdflib.Graph()
    old_g.parse(input_turtle, format="turtle")
    print(f"Loaded graph with {len(old_g)} triples from {input_turtle}")

    # 2) Create a new empty Graph
    new_g = rdflib.Graph()

    # 3) Bind ONLY the single 'ilcd' prefix to your base_uri
    new_g.bind("ilcd", rdflib.URIRef(base_uri))
    # Optionally bind other well-known prefixes like xsd, rdf, skos, etc. if you want
    new_g.bind("xsd", rdflib.URIRef("http://www.w3.org/2001/XMLSchema#"))
    # new_g.bind("skos", rdflib.URIRef("http://www.w3.org/2004/02/skos/core#"))
    # etc.

    def unify_uri(u):
        """
        If `u` is a URIRef matching one of our old namespaces,
        rewrite it to the new base_uri + localname.
        Otherwise, return as-is.
        """
        if not isinstance(u, rdflib.URIRef):
            return u

        # Try computing qname from old_g. This helps us get local name easily.
        # If the URI is completely unknown, we fallback to raw rewriting.
        try:
            prefix, ns, local = old_g.compute_qname(u)
        except Exception:
            # If compute_qname fails, you can keep it as is or do custom logic
            return u

        # Check if the prefix is one of the old ns (ns1, ns2, ILCDex, etc.)
        # or if `ns` matches a known old namespace URI. You can decide your logic here.
        # We'll do a simple approach: if prefix starts with "ns" or in old_ns_list, unify it
        if prefix.startswith("ns") or prefix + ":" in old_ns_list:
            return rdflib.URIRef(base_uri + local)
        if prefix in old_ns_list:
            return rdflib.URIRef(base_uri + local)

        # else keep as-is (e.g., xsd:, rdf:, skos:)
        return u

    # 4) Iterate over all triples in old_g, rewrite them, and add to new_g
    for s, p, o in old_g:
        s_new = unify_uri(s)
        p_new = unify_uri(p)
        o_new = unify_uri(o)
        new_g.add((s_new, p_new, o_new))

    # 5) Serialize the new graph to Turtle
    new_g.serialize(destination=output_turtle, format="turtle")
    print(f"Rewrote {len(new_g)} triples to {output_turtle} with a single 'ilcd:' prefix.")

# -------------------------------------------------------------------------
# USAGE:
unify_ns_prefixes(
    input_turtle="data/rdf/epd_rdf_instance_datastore.ttl",
    output_turtle="data/rdf/epd_rdf_instance_datastore.ttl",
    base_uri="https://example.org/ilcd/",
    old_ns_list=("ILCDex:", "ILCDsd:", "ILCDlcia:", "ILCDmav:", "ILCDadmin:", "ILCDpi:")  # etc.
)


Loaded graph with 99218 triples from data/rdf/epd_rdf_instance_datastore.ttl
Rewrote 99218 triples to data/rdf/epd_rdf_instance_datastore.ttl with a single 'ilcd:' prefix.


In [ ]:
import rdflib
from rdflib import Graph, Namespace, Literal, RDF
from rdflib.namespace import SKOS

# Create a new graph
g = Graph()

# Define unified ILCD namespace and the canonical namespace
ILCD = Namespace("https://example.org/ilcd/")
EX = Namespace("https://example.org/canonical/")

# Bind prefixes
g.bind("ilcd", ILCD)
g.bind("ex", EX)
g.bind("skos", SKOS)

# Load the initial RDF data from file (this file should now use the unified "ilcd:" prefix)
file_path = "data/rdf/epd_rdf_instance_datastore.ttl"
g.parse(file_path, format="turtle")
print(f"Loaded RDF data from {file_path}")

# Define canonical mapping with normalized keys for each category.
# All classification entry literal values are expected to be in the unified namespace context,
# so we use ILCD:value to retrieve their text.
canonical_mapping = {
    "mineral building products": EX.CanonicalCategory_1,
    "mineralische baustoffe": EX.CanonicalCategory_1,
    "mortar and concrete": EX.CanonicalCategory_2,
    "mörtel und beton": EX.CanonicalCategory_2,
    "ready mixed concrete": EX.CanonicalCategory_3,
    "beton": EX.CanonicalCategory_3
}

# Process each classification entry
for entry in g.subjects(RDF.type, ILCD.ClassificationEntry):
    value = g.value(entry, ILCD.value)
    if value is not None:
        norm_value = str(value).lower().strip()
        if norm_value in canonical_mapping:
            canonical_uri = canonical_mapping[norm_value]
            # Add the canonical category triple
            g.add((entry, EX.hasCanonicalCategory, canonical_uri))

# Create canonical category resources with SKOS labels in multiple languages
# Canonical Category 1: Mineral building products / Mineralische Baustoffe
g.add((EX.CanonicalCategory_1, RDF.type, SKOS.Concept))
g.add((EX.CanonicalCategory_1, SKOS.prefLabel, Literal("Mineral building products", lang="en")))
g.add((EX.CanonicalCategory_1, SKOS.prefLabel, Literal("Mineralische Baustoffe", lang="de")))

# Canonical Category 2: Mortar and Concrete / Mörtel und Beton
g.add((EX.CanonicalCategory_2, RDF.type, SKOS.Concept))
g.add((EX.CanonicalCategory_2, SKOS.prefLabel, Literal("Mortar and Concrete", lang="en")))
g.add((EX.CanonicalCategory_2, SKOS.prefLabel, Literal("Mörtel und Beton", lang="de")))

# Canonical Category 3: Ready mixed concrete / Beton
g.add((EX.CanonicalCategory_3, RDF.type, SKOS.Concept))
g.add((EX.CanonicalCategory_3, SKOS.prefLabel, Literal("Ready mixed concrete", lang="en")))
g.add((EX.CanonicalCategory_3, SKOS.prefLabel, Literal("Beton", lang="de")))

print("Canonical SKOS relationships created.")

# Save the enriched graph to a new file with a unified "ilcd:" prefix
output_file_path = "data/rdf/epd_rdf_instance_datastore_canonical_skos.ttl"
g.serialize(destination=output_file_path, format="turtle")
print(f"Graph saved to {output_file_path}")


Loaded RDF data from data/rdf/epd_rdf_instance_datastore.ttl
Canonical SKOS relationships created.
Graph saved to data/rdf/epd_rdf_instance_datastore_canonical_skos.ttl


In [ ]:
import json
import csv
from rdflib import Graph, Namespace, Literal, URIRef
from rdflib.namespace import RDF, SKOS

# Define Namespaces exactly as in your RDF

ILCD = Namespace("https://example.org/ilcd/")
EX = Namespace("https://example.org/canonical/")
DIN = Namespace("https://example.org/din276/")
LINKML_UUID = URIRef("https://w3id.org/linkml/UUIDType")

# Read the EPD Turtle and bind DIN prefix for cleaner RDF
g = Graph()
g.parse("data/rdf/epd_rdf_instance_datastore_canonical_skos.ttl", format="turtle")
g.bind("din", DIN)

# Build dictionary from EPD JSON: epd_id -> epd_uuid
epd_id_to_uuid = {}
with open("../data/pipeline2/json/edited_epds.jsonl", "r", encoding="utf-8") as f:
    for line in f:
        data = json.loads(line.strip())
        epd_id = str(data["id"])
        epd_uuid = data["uuid"]
        epd_id_to_uuid[epd_id] = epd_uuid

# Build dictionary from DIN 276 JSON: custom_id -> [cost_group_codes]
din_cost_groups = {}
used_codes = set()  # collect all cost codes referenced in DIN JSON
with open("../data/pipeline2/json/openai/batch_67d5a00f7f2c8190a0e2cdc3cf04382b_output.jsonl", "r", encoding="utf-8") as f:
    for line in f:
        data = json.loads(line.strip())
        custom_id = str(data["custom_id"])
        content_str = data["response"]["body"]["choices"][0]["message"]["content"]
        parsed = json.loads(content_str)
        cost_codes = parsed["cost_group_codes"]
        din_cost_groups[custom_id] = cost_codes
        used_codes.update(cost_codes)

# Build a SKOS vocabulary for DIN 276 cost groups from CSV,
# but only add rows whose cost code (Nr) is in used_codes.
# Also add the cost group number using skos:notation.
din_code_to_concept = {}
with open("../data/pipeline2/csv/din276_concrete_sub.csv", "r", encoding="utf-8") as csvfile:
    reader = csv.DictReader(csvfile)  # Expected columns: Nr, Cost group (CG), Notes
    for row in reader:
        nr = row["Nr"].strip()  # e.g., "310"
        # Only include cost groups referenced in DIN JSON
        if nr not in used_codes:
            continue

        label_en = row["Cost group (CG)"].strip()  # e.g., "Trenchwork/Earthworks"
        notes_en = row["Notes"].strip()            # e.g., description

        # Create a SKOS concept URI, e.g., DIN:costgroup_310
        concept_uri = DIN[f"costgroup_{nr}"]
        g.add((concept_uri, RDF.type, SKOS.Concept))
        g.add((concept_uri, SKOS.prefLabel, Literal(label_en, lang="en")))
        g.add((concept_uri, SKOS.note, Literal(notes_en, lang="en")))
        # Add the cost group number as a notation
        g.add((concept_uri, SKOS.notation, Literal(nr)))
        din_code_to_concept[nr] = concept_uri

# For each DIN 276 entry in the DIN JSON, find the corresponding top EPD node and link it
# to the SKOS concept(s) for the cost codes.
for custom_id, cost_codes in din_cost_groups.items():
    if custom_id not in epd_id_to_uuid:
        print(f"No EPD found for custom_id: {custom_id}")
        continue
    epd_uuid = epd_id_to_uuid[custom_id]
    uuid_literal = Literal(epd_uuid, datatype=LINKML_UUID)
    
    # Walk from dataSetInformation up to the top EPD node:
    found_epd = None
    for dsi_node in g.subjects(predicate=ILCD.UUID, object=uuid_literal):
        for pi_node in g.subjects(predicate=ILCD.dataSetInformation, object=dsi_node):
            for epd_node in g.subjects(predicate=ILCD.processInformation, object=pi_node):
                found_epd = epd_node
                break
            if found_epd:
                break
        if found_epd:
            break

    if not found_epd:
        print(f"No top EPD node found for UUID: {epd_uuid}")
        continue

    # Only link cost codes that are present (from DIN JSON)
    for code in cost_codes:
        concept_uri = din_code_to_concept.get(code)
        if concept_uri is None:
            # If not found in the CSV vocabulary, skip adding
            print(f"Cost code {code} not in CSV vocabulary; skipping.")
            continue
        # Link using the DIN namespace property for clarity.
        g.add((found_epd, DIN.hasDIN276CostGroup, concept_uri))
    print(f"Linked EPD node {found_epd} to cost codes: {cost_codes}")

# Add broader/narrower relationships among cost group concepts.
# Example rule: if a code ends with "0" and another code shares the same first two characters,
# then the one ending with "0" is broader than the other.
for code_parent, concept_parent in din_code_to_concept.items():
    # Check that parent candidate ends with "0" (broader category)
    if not code_parent.endswith("0"):
        continue
    for code_child, concept_child in din_code_to_concept.items():
        if code_child == code_parent:
            continue
        # If the first two characters match and the child code does not end with "0"
        if code_child.startswith(code_parent[:2]) and not code_child.endswith("0"):
            # Add bidirectional broader/narrower relationships
            g.add((concept_child, SKOS.broader, concept_parent))
            g.add((concept_parent, SKOS.narrower, concept_child))

# Serialize updated RDF
g.serialize(destination="data/rdf/epd_rdf_instance_datastore_canonical_skos_din.ttl", format="turtle")
print("Done! Updated RDF with DIN 276 SKOS cost groups and hierarchical relationships.")


Linked EPD node https://example.org/ilcd/63a79af11ab0467745a808dc6fc9d4ca to cost codes: ['320', '322', '350', '351', '360', '361']
Linked EPD node https://example.org/ilcd/46d205a58dcd4d37c91008dc77c8811e to cost codes: ['320', '322', '350', '351']
Linked EPD node https://example.org/ilcd/e79a42aa51cb4d28230808dc685f6882 to cost codes: ['320', '322', '330', '331', '340', '341', '350', '351', '360', '361']
Linked EPD node https://example.org/ilcd/32426332ecca4bd91d3f08dca1505d10 to cost codes: ['320', '322', '330', '331', '340', '341', '350', '351', '360', '361']
Linked EPD node https://example.org/ilcd/4f634da7bbe1421d231008dc685f6882 to cost codes: ['320', '322', '350', '351']
Linked EPD node https://example.org/ilcd/14358dd4831f40abd98d08dd1f3f8b97 to cost codes: ['320', '322', '350', '351']
Linked EPD node https://example.org/ilcd/827fdb940b814936231c08dc685f6882 to cost codes: ['320', '322', '340', '341', '350', '351', '360', '361']
Linked EPD node https://example.org/ilcd/bbc4826

In [7]:
from rdflib import Graph, Namespace, URIRef, Literal, BNode
from rdflib.namespace import RDF, RDFS, SKOS, XSD
import xml.etree.ElementTree as ET

# Namespaces used in your KG and for BKI data
BKI = Namespace("https://example.org/bki#")  # for your BKI elements
DIN = Namespace("https://example.org/din276/")  # existing DIN cost groups

# Load your existing SKOS-based DIN 276 cost-group ontology
# The triple store includes definitions like din:costgroup_322, etc.

g = Graph()
g.parse("data/rdf/epd_rdf_instance_datastore_canonical_skos_din.ttl", format="turtle")

# Register custom namespaces in the graph for clarity
g.bind("bki", BKI)
g.bind("din", DIN)

def integrate_bki_xml(g, xml_file_path):
    """
    Parse a BKI XML file (level 2 or 3),
    create RDF triples for each <element> that has a component containing 'transportbeton',
    and link them to the DIN 276 cost groups in the existing graph.
    """

    print(f"Starting graph size: {len(g)} triples.")
    
    # Register namespace to avoid parser warnings
    ET.register_namespace('', 'https://www.bauteileditor.de')
    
    tree = ET.parse(xml_file_path)
    root = tree.getroot()
    
    # Define XML namespaces
    ns = {'elca': 'https://www.bauteileditor.de'}

    # Iterate over all element nodes
    for element_node in root.findall('.//elca:element', ns):
        component_nodes = element_node.findall('.//elca:component', ns)
        # Check if any component contains 'transportbeton'
        has_transportbeton = any('transportbeton' in comp.get('processConfigName', '').lower() for comp in component_nodes)
        
        if not has_transportbeton:
            continue  # Skip elements without 'transportbeton' in components

        # Extract the element's UUID and DIN code
        element_uuid = element_node.get('uuid', 'unknown').replace('-', '')
        din_code = element_node.get('din276Code', 'unknown')
        element_uri = BKI[f'element_{element_uuid}']

        # Add RDF triples to the graph
        g.add((element_uri, RDF.type, BKI.BKIElement))
        costgroup_uri = DIN[f'costgroup_{din_code}']
        g.add((element_uri, DIN.hasDIN276CostGroup, costgroup_uri))

        # Extract name and description from <elementInfo>
        elem_info = element_node.find('.//elca:elementInfo', ns)
        if elem_info is not None:
            name_node = elem_info.find('elca:name', ns)
            desc_node = elem_info.find('elca:description', ns)
            
            if name_node is not None and name_node.text:
                g.add((element_uri, BKI.name, Literal(name_node.text.strip(), datatype=XSD.string)))
            if desc_node is not None and desc_node.text:
                g.add((element_uri, BKI.description, Literal(desc_node.text.strip(), datatype=XSD.string)))

        # Extract details for each component layer
        for comp_node in component_nodes:
            process_config_uuid = comp_node.get('processConfigUuid', '').replace('-', '')
            layer_size = comp_node.get('layerSize', '')
            layer_size_str = layer_size.replace('.', '').ljust(3, '0')[:3]  # e.g., "0.15" -> "015"
            layer_uri = BKI[f'layer_{process_config_uuid}_{layer_size_str}']

            process_config_name = comp_node.get('processConfigName', '')
            life_time = comp_node.get('lifeTime', '')
            
            g.add((layer_uri, RDF.type, BKI.Layer))
            g.add((element_uri, BKI.hasLayer, layer_uri))
            g.add((layer_uri, BKI.processConfigName, Literal(process_config_name, datatype=XSD.string)))
            
            if life_time.isdigit():
                g.add((layer_uri, BKI.lifeTime, Literal(int(life_time), datatype=XSD.integer)))
            else:
                g.add((layer_uri, BKI.lifeTime, Literal(life_time, datatype=XSD.string)))
            
            try:
                layer_size_float = float(layer_size)
                g.add((layer_uri, BKI.layerSize, Literal(layer_size_float, datatype=XSD.float)))
            except ValueError:
                g.add((layer_uri, BKI.layerSize, Literal(layer_size, datatype=XSD.string)))

    print(f"Integrated data from {xml_file_path}. Current graph size: {len(g)} triples.")

integrate_bki_xml(g, "../data/pipeline2/xml/BKI_Bauteilaufbauten_ 2_Ebene_DIN_276.xml")
integrate_bki_xml(g, "../data/pipeline2/xml/BKI_Bauteilaufbauten_ 3_Ebene_DIN_276.xml")

Starting graph size: 95125 triples.
Integrated data from ../data/pipeline2/xml/BKI_Bauteilaufbauten_ 2_Ebene_DIN_276.xml. Current graph size: 95522 triples.
Starting graph size: 95522 triples.
Integrated data from ../data/pipeline2/xml/BKI_Bauteilaufbauten_ 3_Ebene_DIN_276.xml. Current graph size: 95735 triples.


In [8]:
# Serialize updated RDF
g.serialize(destination="data/rdf/epd_rdf_instance_datastore_canonical_skos_din_bki.ttl", format="turtle")
print("Done! Updated RDF with BKI elements.")

Done! Updated RDF with BKI elements.


In [6]:
def query_bki_by_costgroups(g, costgroup_notations):
    """
    Returns all BKI elements linked to a DIN 276 cost group whose notation 
    is in costgroup_notations, and also whose layers contain 'beton'.
    """
    # Build the string list for SPARQL: e.g. ("322","330","340")
    notations_list = '","'.join(costgroup_notations)  # e.g. '322","330","340'
    
    # For the SPARQL, we can do: FILTER(?notation IN ("322","330","340"))
    # We'll avoid backslashes around quotes by using triple single-quotes:
    
    query_str = f'''
PREFIX ex: <{EX}>
PREFIX bki: <{BKI}>
PREFIX din: <{DIN}>
PREFIX skos: <{SKOS}>

SELECT ?element ?name ?description ?layerName ?layerLife ?layerSize ?notation
WHERE {{
  ?element a ex:BKIElement ;
           din:hasDIN276CostGroup ?cg ;
           ex:name ?name ;
           ex:description ?description ;
           ex:hasLayer ?layer .
  
  ?layer ex:processConfigName ?layerName .
  OPTIONAL {{ ?layer ex:lifeTime ?layerLife . }}
  OPTIONAL {{ ?layer ex:layerSize ?layerSize . }}
  
  ?cg skos:notation ?notation .
  
  FILTER(?notation IN ("{notations_list}"))
  FILTER regex(?layerName, "beton", "i")
}}
ORDER BY ?element
'''
    
    results = g.query(query_str)
    print(query_str)
    
    # Convert results to a Python structure
    from collections import defaultdict
    
    # Dictionary keyed by element URI
    output = defaultdict(lambda: {
        'name': '',
        'description': '',
        'notation': '',
        'layers': []
    })
    
    for row in results:
        elem_uri = str(row.element)
        
        if not output[elem_uri]['name']:
            output[elem_uri]['name'] = str(row.name)
        if not output[elem_uri]['description']:
            output[elem_uri]['description'] = str(row.description)
        if not output[elem_uri]['notation']:
            output[elem_uri]['notation'] = str(row.notation)
        
        # Layers
        layer_data = {
            'processConfigName': str(row.layerName),
            'lifeTime': str(row.layerLife) if row.layerLife else None,
            'layerSize': str(row.layerSize) if row.layerSize else None
        }
        output[elem_uri]['layers'].append(layer_data)
    
    return output


result = query_bki_by_costgroups(g, ["322","330","331","340"])
for elem_uri, info in result.items():
    print(f"Element: {elem_uri}")
    print(f"  DIN 276 group notation: {info['notation']}")
    print(f"  Name: {info['name']}")
    print(f"  Description: {info['description']}")
    print("  Layers containing 'beton':")
    for layer in info['layers']:
        print(f"    * processConfigName: {layer['processConfigName']}")
        print(f"      lifeTime: {layer['lifeTime']}")
        print(f"      layerSize: {layer['layerSize']}")
    print("---")



PREFIX ex: <https://example.org/ex#>
PREFIX bki: <https://example.org/bki#>
PREFIX din: <https://example.org/din276/>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>

SELECT ?element ?name ?description ?layerName ?layerLife ?layerSize ?notation
WHERE {
  ?element a ex:BKIElement ;
           din:hasDIN276CostGroup ?cg ;
           ex:name ?name ;
           ex:description ?description ;
           ex:hasLayer ?layer .
  
  ?layer ex:processConfigName ?layerName .
  OPTIONAL { ?layer ex:lifeTime ?layerLife . }
  OPTIONAL { ?layer ex:layerSize ?layerSize . }
  
  ?cg skos:notation ?notation .
  
  FILTER(?notation IN ("322","330","331","340"))
  FILTER regex(?layerName, "beton", "i")
}
ORDER BY ?element

Element: https://example.org/bki#element_04bc2aba-59b7-4ed9-a4d4-d90619d57547
  DIN 276 group notation: 331
  Name: 054 AW KG, Stahlbeton, WU, 30cm, Bewehrung 120kg/m3, Schalung glatt
  Description: Außenwand der Kellerwand, aus Stahlbeton C30/37, wasserundurchlässig, Bewehrung 120 

# Helpers

In [8]:
import rdflib

def extract_related_triples(graph, node, visited, subgraph):
    """
    Recursively extracts all triples starting from 'node' into 'subgraph'.
    Prevents cycles using the 'visited' set.
    """
    if node in visited:
        return
    visited.add(node)
    for predicate, obj in graph.predicate_objects(subject=node):
        subgraph.add((node, predicate, obj))
        # Traverse further if the object is a URIRef or Blank Node.
        if isinstance(obj, (rdflib.URIRef, rdflib.BNode)):
            extract_related_triples(graph, obj, visited, subgraph)

def extract_entity_subgraph(file_path, entity_uri):
    """
    - Loads a Turtle file.
    - Extracts the subgraph related to the given 'entity_uri' (following outgoing triples).
    - Returns the extracted subgraph serialized in Turtle format.
    """
    g = rdflib.Graph()
    g.parse(file_path, format="turtle")
    
    target_node = rdflib.URIRef(entity_uri)
    if target_node not in set(g.subjects()):
        print(f"Entity {entity_uri} not found as a subject in the graph.")
        return None
    
    # Create a new graph for the subgraph and use the original's namespace manager.
    subgraph = rdflib.Graph()
    subgraph.namespace_manager = g.namespace_manager
    
    visited = set()
    extract_related_triples(g, target_node, visited, subgraph)
    
    turtle_output = subgraph.serialize(format="turtle")
    # Decode if the output is in bytes.
    if isinstance(turtle_output, bytes):
        turtle_output = turtle_output.decode("utf-8")
    return turtle_output

# Example usage in Jupyter Notebook:
entity_turtle = extract_entity_subgraph(
    'data/rdf/epd_rdf_instance_datastore_canonical_skos_din.ttl',
    'https://example.org/ilcd/0643c04910464d518ddc760f32642a72'
)
print(entity_turtle)


@prefix din: <https://example.org/din276/> .
@prefix ex: <https://example.org/canonical/> .
@prefix ilcd: <https://example.org/ilcd/> .
@prefix skos: <http://www.w3.org/2004/02/skos/core#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

ilcd:0643c04910464d518ddc760f32642a72 a ilcd:ProcessDataSet ;
    din:hasDIN276CostGroup din:costgroup_320,
        din:costgroup_322,
        din:costgroup_330,
        din:costgroup_331,
        din:costgroup_340,
        din:costgroup_341,
        din:costgroup_350,
        din:costgroup_351,
        din:costgroup_360,
        din:costgroup_361,
        din:costgroup_370,
        din:costgroup_371 ;
    ilcd:administrativeInformation ilcd:0643c04910464d518ddc760f32642a72_administrativeInformation ;
    ilcd:exchanges ilcd:0643c04910464d518ddc760f32642a72_exchanges ;
    ilcd:lciaResults ilcd:0643c04910464d518ddc760f32642a72_lciaResults ;
    ilcd:modellingAndValidation ilcd:0643c04910464d518ddc760f32642a72_modellingAndValidation ;
    ilcd:proc